<a href="https://colab.research.google.com/github/Murcha1990/ML_AI24/blob/main/Hometasks/Base/AI_HW6_uplift.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1> Задание по Uplift-моделированию </h1>

<h2>Введение</h2>

Перед вами типичная задача, возникающая при работе с моделями кампейнинга в банке: заказчик запустил несколько пилотов по взаимодействию с клиентами с помощью разных каналов: push в мобильном приложении, sms, баннеры в мобильном приложении и реклама в других приложениях экосистемы. Заказчик хотел бы понимать, какой канал взаимодействия с клиентом наиболее эффективен для каждого клиента из клиентской базы. Кампании планируются и запускаются в ежемесячном режиме. Иными словами, заказчик хотел бы в идеале ежемесячно получать список клиентов, которым необходимо отправить коммуникацию с указанием канала и прироста вероятности покупки в случае, если клиенту отправят коммуникацию по сравнению с тем случаем, когда клиенту коммуникацию не отправят.

<b>Таким образом: </b>
1.	У нас есть база клиентов (клиенты, имеющие id в банке). По данной базе осуществляется рассылка тех или иных стимулирующих коммуникаций по различным продуктам, каналам (например SMS, Push, баннеры в мобильном приложении и т.д.) и сегментам клиентов
2.	Признаковое описание клиента состоит из различных агрегатов действий клиента за месяц или его объективных характеристик: например, средняя сумма средств на депозитах за месяц, среднее число кликов клиента в день за месяц в разделе "инвестиции" в мобильном приложении или возраст клиента
3.	При формировании обучающей/тестовой выборки допускается, что один и тот же клиент за разные месяцы — это разные объекты. То есть допускается, что клиент в феврале и клиент в марте — это разные клиенты (то есть мы можем оперировать с ними как с разными сущностями).
4.	Агрегаты действий клиента за месяц появляются примерно 10 числа следующего месяца. То есть, например, агрегаты за декабрь появляются 10 января. В свою очередь списки клиентов, которым необходимо осуществить рассылку должны быть сформированы ориентировочно 20 числа предыдущего месяца. Таким образом, <b> модель должна быть обучена делать предсказания с лагом в два месяца </b>, то есть должна делать предсказание на март по клиентским агрегатам за январь. Обязательно учтите это при обучении модели (в противном случае можно получить лик таргета, так как часто величину, которую мы предсказываем уже есть в клиентских агрегатах, но смещенная на два месяца).


## Оценивание задания:

Всего за задание можно получить 50 первичных баллов, которые затем переводятся в 10-балльную шкалу делением не 5.

Скачаем архив с данными по ссылке и разархивируем.

In [1]:
!pip install gdown -q

In [2]:
import gdown

url = 'https://drive.google.com/uc?id=19nKGaxm3RwHxh2UWPo537_-MDx21AkHO'
output = 'Data.zip'
gdown.download(url, output, quiet=False)

Downloading...
From (original): https://drive.google.com/uc?id=19nKGaxm3RwHxh2UWPo537_-MDx21AkHO
From (redirected): https://drive.google.com/uc?id=19nKGaxm3RwHxh2UWPo537_-MDx21AkHO&confirm=t&uuid=be25b41d-6552-418c-8fb5-2e6ed5b24ff9
To: /content/Data.zip
100%|██████████| 289M/289M [00:09<00:00, 29.1MB/s]


'Data.zip'

In [3]:
import zipfile

with zipfile.ZipFile('Data.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/')

<h2>Описание данных</h2>

Перед вами несколько наборов данных, на основе которых вам будет необходимо обучить Uplift модели, сделать прогноз на нужный месяц и решить, кому из клиентов отправлять коммуникацию, а кому коммуникацию отправлять не следует.

<h3>Features </h3> Признаки клиентов, клиентские агрегаты, которые описывают поведение клиентов <br>

1. user_id - id клиента
2. report_dt - месяц, на который актуальны признаки
3. city - город, в котором живет клиент
4. age - возраст клиента
5. x1 – x9 - числовые признаки клиента, характеризующие поведение клиента

Первичный ключ таблицы - user_id + report_dt

<h3> Contracts </h3> Таблица с покупками продуктов.

1. contract_id - id покупки
2. user_id - id пользователя, который совершил покупку
3. product_id - id продукта, который был куплен
4. contract_ts – дата момента, когда была совершена покупка

Первичный ключ - contract_id


<h3> Campaings </h3> Кампании, которые проводились (под кампанией мы понимаем рассылку sms, push и т.д).

1. campaing_id - id кампании, первичный ключ таблицы
2. product_id - продукт, по которому проводилась кампания (считаем, что продукты не конкурируют друг с другом)
3. channel - канал, в котором проводилась кампания


<h3> People_in_campaings </h3> Люди, которые принимали участие в кампаниях.

1. campaing_id - id кампании
2. user_id - id пользователя, который попал в кампанию
3. флаг целевой (1) и контрольной (0) группы (целевая группа - это те, кто получил коммуникацию, а контрольная - те, кто нет)
4. delivery_ts - timestamp, когда клиенту фактически была доставлена коммуникация (для контрольной группы nan, подумайте почему)

Первичный ключ данной таблицы - user_id + campaing_id


<h3> Contracts </h3> Таблица с покупками продуктов

1. contract_id - id покупки
2. user_id - id пользователя, который совершил покупку
3. product_id - id продукта, который был куплен
4. contract_ts – дата момента, когда была совершена покупка

Первичный ключ - contract_id


<h1> Постановка задачи </h1> В ноябре 2024 проводилось несколько кампаний по продукту с id 0001 (фактически клиенту рассылалось одно и тоже сообщение, но в разных каналах). Вам необходимо по данным кампаниям построить модель, которая будет определять лучший канал коммуникации каждого клиента и определить, кому из клиентов в марте 2025 отправить какую коммуникацию, а кому коммуникацию вообще отправлять не следует.
Ответ нужно представить в следующем виде (report_dt – дата фичей):

<table>
  <thead>
    <tr>
      <th>user_id</th>
      <th>report_dt</th>
      <th>channel</th>
      <th>uplift</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>10045</td>
      <td>2025-01-31</td>
      <td>banner</td>
      <td>0.07</td>
    </tr>
    <tr>
      <td>10046</td>
      <td>2025-01-31</td>
      <td>no_comm</td>
      <td>0.00</td>
    </tr>
    <tr>
      <td>10047</td>
      <td>2025-01-31</td>
      <td>sms</td>
      <td>0.23</td>
    </tr>
    <tr>
      <td>10048</td>
      <td>2025-01-31</td>
      <td>push</td>
      <td>0.19</td>
    </tr>
  </tbody>
</table>

<h1> Декомпозиция задачи </h1>

<h2> 1.	Сбор и анализ таргета (18 баллов)</h2>

Прежде всего, вам необходимо собрать целевое событие, которое вы собираетесь прогнозировать. В данном случае целевое событие - это покупка продукта 0001 пользователем, участвовавшем в кампании. Обратите внимание, что не все пользователи получают коммуникацию одновременно (delivery_ts в таблице People_in_campaings). Согласно правилу, согласованному с заказчиком, <b> человек из целевой группы купил продукт после коммуникации - это значит, что он купил его в течение 2х недель после получения сообщения, а человек из контрольной - в течение 3х недель с момента старта кампании (старт кампании - начало месяца). </b> То есть для определенной кампании, для каждого клиента, попавшего в кампанию, вам надо будет найти его покупки данного продукта, а потом основываяся на данном правиле превратить покупки в 0 или 1. <br> На выходе у вас должен появиться таблица с целевым действием для каждого канала (колонки client_id, report_dt,  target), где таргет - это бинарная переменная (0 или 1). Колонка report_dt вам нужна как техническая колонка для дальнейших джоинов.<br><br>

Проведите анализ полученных данных (до присоединения клиентских агрегатов). Какие проблемы и сложности в данных вы обнаружили? Что с ними можно сделать? Какая из кампаний наиболее эффективная? Подготовьте выводы по полученным инсайтам.


**Комментарий по заданиям и оцениванию:**

* Вы должны самостоятельно сделать join нескольких таблиц, самостоятельно собрать целевое действие

* Представлены 4 различных канала, за таргет по каждому из каналов можно получить **максимум 2 балла**:
    * 1 балл за то, что просчитано целевое действие для целевой группы (покупка в
течение одной-двух недель с момента получения коммуникации)
    * 1 балл за то, что просчитано целевое действие для контрольной группы (покупка в течение двух-трех недель с момента старта кампании) и сделана таблица в требуемом формате

* Обратите внимание, что не во всех кампаниях содержатся корректные данные для проведения моделирования, и вам необходимо провести анализ данных и в случае выявленных некорректностей - описать их, и не проводить моделирование для "сломанной" кампании  
    * За данный анализ можно получить **8 баллов**

* Вы должны оценить эффективность кампаний по uplift (cреднее значение таргета в целевой минус среднее значение таргета в контрольной группе)
    * За данный анализ можно получить **2 балла**

In [ ]:
import os

print("Файлы в /content/:", os.listdir("/content/"))

if "ДЗ по Uplift обновленное" in os.listdir("/content/"):
    print("Файлы в папке 'ДЗ по Uplift обновленное':", os.listdir("/content/ДЗ по Uplift обновленное/"))

Файлы в /content/: ['.config', 'Data.zip', 'ДЗ по Uplift обновленное', 'sample_data']
Файлы в папке 'ДЗ по Uplift обновленное': ['CAMPAINGS.csv', 'AGGS_FINAL.csv', 'PEOPLE_IN_CAMPAINGS_FINAL.csv', 'CONTRACTS_FINAL.csv']


In [ ]:
import pandas as pd

features = pd.read_csv('/content/ДЗ по Uplift обновленное/AGGS_FINAL.csv')
contracts = pd.read_csv('/content/ДЗ по Uplift обновленное/CONTRACTS_FINAL.csv')
campaigns = pd.read_csv('/content/ДЗ по Uplift обновленное/CAMPAINGS.csv')
people_in_campaigns = pd.read_csv('/content/ДЗ по Uplift обновленное/PEOPLE_IN_CAMPAINGS_FINAL.csv')

print("Features:")
print(features.head())

print("\nContracts:")
print(contracts.head())

print("\nCampaigns:")
print(campaigns.head())

print("\nPeople in Campaigns:")
print(people_in_campaigns.head())

Features:
   Unnamed: 0        x1        x2        x3        x4        x5        x6  \
0      104548  0.654343 -1.439286 -0.011475  2.039457  0.843580 -0.977480   
1       38396  2.583579  1.755569  3.360186 -1.122864  0.034201 -0.269607   
2      227077  0.296030 -0.937075  1.073280  1.874636 -0.981216 -1.100187   
3      304649  2.329328 -1.345159  0.345066  0.755373 -0.082842  0.028439   
4      239518  0.167643  1.587099  0.165357  0.289758 -1.108840 -1.501819   

         x7        x8        x9   report_dt  user_id  age    city  
0 -0.768019 -1.044127  0.025673  2025-01-31  1066338   26     Ufa  
1 -1.503646  1.040289 -1.691606  2024-11-30    13900   35     Ufa  
2 -0.331181 -1.575637  0.474965  2025-03-31  4063636   28     Ufa  
3  0.919211  0.808793 -0.560004  2025-03-31  1025488   27  Moscow  
4  0.615588  1.631203 -0.208419  2025-02-28  4040555   37  Moscow  

Contracts:
   Unnamed: 0  user_id contract_date  product_id              contract_id
0       39735  4008279    2024-11

In [ ]:
contracts['contract_date'] = pd.to_datetime(contracts['contract_date'], errors='coerce')
people_in_campaigns['delivery_date'] = pd.to_datetime(people_in_campaigns['delivery_date'], errors='coerce')

In [ ]:
contracts['contract_date'] = pd.to_datetime(contracts['contract_date'], errors='coerce')
people_in_campaigns['delivery_date'] = pd.to_datetime(people_in_campaigns['delivery_date'], errors='coerce')

control_group = people_in_campaigns['t_flag'] == 0
people_in_campaigns.loc[control_group, 'start_date'] = pd.to_datetime('2024-11-01')
people_in_campaigns.loc[control_group, 'end_date'] = pd.to_datetime('2024-11-01') + pd.Timedelta(days=21)

treatment_group = people_in_campaigns['t_flag'] == 1
people_in_campaigns.loc[treatment_group, 'start_date'] = people_in_campaigns['delivery_date']
people_in_campaigns.loc[treatment_group, 'end_date'] = people_in_campaigns['delivery_date'] + pd.Timedelta(days=14)

contracts_filtered = contracts[contracts['product_id'] == 1] 

merged = people_in_campaigns.merge(
    contracts_filtered[['user_id', 'contract_date']],
    on='user_id',
    how='left'
)

merged['target'] = (merged['contract_date'] >= merged['start_date']) & (merged['contract_date'] <= merged['end_date'])
merged['target'] = merged['target'].astype(int)1

people_in_campaigns = merged[['user_id', 'campaing_id', 't_flag', 'delivery_date', 'target']]

print(people_in_campaigns.head())

   user_id campaing_id  t_flag delivery_date  target
0  1099975      idclip       1    2024-11-06       1
1     1162       iddqd       1    2024-11-08       1
2    42991       iddqd       1    2024-11-07       0
3   142343      idclip       0           NaT       1
4    24623       iddqd       0           NaT       0


In [ ]:
campaign_groups = people_in_campaigns.groupby('campaing_id')['t_flag'].nunique()

invalid_campaigns = campaign_groups[campaign_groups < 2].index.tolist()

if invalid_campaigns:
    print("Эти кампании содержат только одну группу (невалидны):", invalid_campaigns)
else:
    print("Во всех кампаниях есть и контрольная, и целевая группы.")

Во всех кампаниях есть и контрольная, и целевая группы.


In [ ]:
campaign_counts = people_in_campaigns['campaing_id'].value_counts()

low_data_campaigns = campaign_counts[campaign_counts < 100].index.tolist()

if low_data_campaigns:
    print("Эти кампании имеют слишком мало данных:", low_data_campaigns)
else:
    print("Все кампании имеют достаточно данных.")

Все кампании имеют достаточно данных.


In [ ]:
campaign_target_ratios = people_in_campaigns.groupby('campaing_id')['target'].mean()

threshold = 0.05 
suspicious_campaigns = campaign_target_ratios[
    (campaign_target_ratios < threshold) | (campaign_target_ratios > (1 - threshold))
].index.tolist()

if suspicious_campaigns:
    print("Эти кампании выглядят странно (почти все target = 0 или 1):", suspicious_campaigns)
else:
    print("Все кампании выглядят нормально.")

Все кампании выглядят нормально.


In [ ]:
people_in_campaigns = people_in_campaigns.merge(campaigns[['campaing_id', 'channel']], on='campaing_id', how='left')

uplift_data = people_in_campaigns.groupby(['channel', 't_flag'])['target'].mean().unstack()

uplift_data['uplift'] = uplift_data[1] - uplift_data[0]

print("Uplift по каналам:")
print(uplift_data)

Uplift по каналам:
t_flag            0         1    uplift
channel                                
banner     0.400733  0.602717  0.201983
other_ads  0.400733  0.602717  0.201983
push       0.202150  0.601738  0.399588
sms        0.684917  0.201167 -0.483750


у смс отрицательный аплифт, это не есть хорошо, надо разобраться

In [ ]:
sms_data = people_in_campaigns[people_in_campaigns['channel'] == 'sms']
sms_target_rates = sms_data.groupby('t_flag')['target'].mean()

print("Средний target по SMS:\n", sms_target_rates)

Средний target по SMS:
 t_flag
0    0.684917
1    0.201167
Name: target, dtype: float64


In [ ]:
print("Минимальная дата рассылки:", people_in_campaigns['delivery_date'].min())
print("Максимальная дата рассылки:", people_in_campaigns['delivery_date'].max())

Минимальная дата рассылки: 2024-11-04 00:00:00
Максимальная дата рассылки: 2024-11-08 00:00:00


In [ ]:
features_unique = features[['user_id', 'city']].drop_duplicates(subset=['user_id'])

city_distribution = people_in_campaigns['user_id'].map(features_unique.set_index('user_id')['city']).value_counts()

print("Распределение пользователей по городам:\n", city_distribution)

Распределение пользователей по городам:
 user_id
Smolensk    189210
Ufa         165961
Moscow      164829
Name: count, dtype: int64


средний таргет для контрольной группы выше чем у тех кто получил смс, делаем вывод что смс сломана

In [ ]:
people_in_campaigns_cleaned = people_in_campaigns[people_in_campaigns['channel'] != 'sms']

print("Оставшиеся каналы после очистки:", people_in_campaigns_cleaned['channel'].unique())

Оставшиеся каналы после очистки: ['push' 'banner' 'other_ads']


In [ ]:
uplift_data_cleaned = people_in_campaigns_cleaned.groupby(['channel', 't_flag'])['target'].mean().unstack()
uplift_data_cleaned['uplift'] = uplift_data_cleaned[1] - uplift_data_cleaned[0]

print("Обновленный Uplift по каналам (без SMS):")
print(uplift_data_cleaned)


Обновленный Uplift по каналам (без SMS):
t_flag            0         1    uplift
channel                                
banner     0.400733  0.602717  0.201983
other_ads  0.400733  0.602717  0.201983
push       0.202150  0.601738  0.399588


### ваши выводы здесь

<h2> 2. Клиентские агрегаты (12 баллов)</h2>

Присоедините клиентские агрегаты (будьте внимательны, присоедините агрегаты за корректный месяц) и изучите полученные данные.

**Комментарий по заданиям и оцениванию:**

* Вы должны корректно присоединить клиентские агрегаты со смещением на два месяца, чтобы не было лика таргета. За данное действие можно получить **4 балла**

* Далее вы должен сделать UPLIFT EDA, которые обсуждались на лекции и показывались в практических ноутбуках. В ходе анализа вы должны проверить корректность данных по рекламным кампаниям и решить, что делать со "сломанными" кампаниями. По итогам анализа подготовьте выводы. За данное действие можно получить **8 баллов**

In [ ]:
features['report_dt'] = pd.to_datetime(features['report_dt'])
people_in_campaigns_cleaned['delivery_date'] = pd.to_datetime(people_in_campaigns_cleaned['delivery_date'])

people_in_campaigns_cleaned['report_dt'] = people_in_campaigns_cleaned['delivery_date'] - pd.DateOffset(months=2)

data_merged = people_in_campaigns_cleaned.merge(
    features,
    on=['user_id', 'report_dt'],
    how='left'
)

print("Размер объединенных данных:", data_merged.shape)

print("Количество пропусков после объединения:\n", data_merged.isnull().sum())


<ipython-input-20-df26d81869e0>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  people_in_campaigns_cleaned['delivery_date'] = pd.to_datetime(people_in_campaigns_cleaned['delivery_date'])
<ipython-input-20-df26d81869e0>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  people_in_campaigns_cleaned['report_dt'] = people_in_campaigns_cleaned['delivery_date'] - pd.DateOffset(months=2)


Размер объединенных данных: (400000, 19)
Количество пропусков после объединения:
 user_id               0
campaing_id           0
t_flag                0
delivery_date    200000
target                0
channel               0
report_dt        200000
Unnamed: 0       400000
x1               400000
x2               400000
x3               400000
x4               400000
x5               400000
x6               400000
x7               400000
x8               400000
x9               400000
age              400000
city             400000
dtype: int64


In [ ]:
import numpy as np

people_in_campaigns_cleaned['delivery_date'] = pd.to_datetime(people_in_campaigns_cleaned['delivery_date'])

people_in_campaigns_cleaned.loc[people_in_campaigns_cleaned['delivery_date'].notna(), 'report_dt'] = (
    people_in_campaigns_cleaned['delivery_date'] - pd.DateOffset(months=2)
).dt.to_period('M').dt.to_timestamp(how='end')

people_in_campaigns_cleaned.loc[people_in_campaigns_cleaned['delivery_date'].isna(), 'report_dt'] = pd.Timestamp('2024-09-30')

people_in_campaigns_cleaned['report_dt'] = pd.to_datetime(people_in_campaigns_cleaned['report_dt'])

data_merged = people_in_campaigns_cleaned.merge(
    features,
    on=['user_id', 'report_dt'],
    how='left'
)

print("Пропуски после объединения:\n", data_merged.isnull().sum())


<ipython-input-26-f42398d0e85b>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  people_in_campaigns_cleaned['delivery_date'] = pd.to_datetime(people_in_campaigns_cleaned['delivery_date'])
<ipython-input-26-f42398d0e85b>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  people_in_campaigns_cleaned['report_dt'] = pd.to_datetime(people_in_campaigns_cleaned['report_dt'])


Пропуски после объединения:
 user_id               0
campaing_id           0
t_flag                0
delivery_date    200000
target                0
channel               0
report_dt             0
Unnamed: 0       200000
x1               200000
x2               200000
x3               200000
x4               200000
x5               200000
x6               200000
x7               200000
x8               200000
x9               200000
age              200000
city             200000
dtype: int64


In [ ]:
data_cleaned = data_merged.dropna(subset=['x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8', 'x9', 'age', 'city'])

print("Размер очищенных данных:", data_cleaned.shape)

print("Пропуски после удаления пустых строк:\n", data_cleaned.isnull().sum())


Размер очищенных данных: (200000, 19)
Пропуски после удаления пустых строк:
 user_id               0
campaing_id           0
t_flag                0
delivery_date    200000
target                0
channel               0
report_dt             0
Unnamed: 0            0
x1                    0
x2                    0
x3                    0
x4                    0
x5                    0
x6                    0
x7                    0
x8                    0
x9                    0
age                   0
city                  0
dtype: int64


### ваши выводы здесь

<h2> 3. Построение моделей и оценка их качества (14 баллов)</h2>

Постройте Uplift модели по собранным кампаниям, проведите тюнинг гиперпараметров и оцените их качество (qini score). Для каждой модели также постройте qini-curve.

<h2>4. Подготовка ответа в требуемом формате и подготовка выводов (6 баллов)</h2>

a) Сделайте скоринг нужных клиентов, подготовьте ответ в требуемом формате

б) Сделайте краткую аналитику того, какой канал взаимодействия наиболее предпочтителен

в) Сделайте выводы по проделанной работе

**Комментарий по заданиям и оцениванию:**

* Подготовлен только ответ - **1 балл**
* Подготовлен содержательный вывод по проделанной работе - **4 балла**
* Корректно принято решение об отправке/не отправке коммуникации клиентам в зависимости от значений Uplift - **1 балл**

In [18]:
# ваш код здесь

### ваши выводы здесь